In [ ]:
import numpy as np
import pandas as pd
from pandas import Series
from numpy import float32
import math
from numpy import inf
import warnings
import time
import os
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

In [ ]:
expression=np.loadtxt('模块初筛后基因表达数据.csv',dtype=float32,delimiter=",")
feature = np.array(expression[:,:-1])
labels = np.array(expression[:,-1],dtype=int)

In [ ]:
def feature_ranking(F):
    idx = np.argsort(F)   
    return idx[::-1]    

In [ ]:
acc_list_1 =list()
def main1():
    expression=np.loadtxt('模块初筛后基因表达数据.csv',delimiter=',')
    X=np.array(expression[:,:-1],dtype=float32)
    y=np.array(expression[:,-1], dtype=int)
    n_samples, n_features = X.shape    
    kf =KFold(n_splits=10,random_state=0)
    score_1 =np.loadtxt('generank分数.csv',delimiter=',')
    score_1 = np.array(score_1,dtype=float32)
    score_1 = (score_1-min(score_1))/(max(score_1)-min(score_1))
    score_2 = np.loadtxt('nn分数.csv',delimiter=',') 
    score_2 = np.array(score_2,dtype=float32)
    score_2 = (score_2-min(score_2))/(max(score_2)-min(score_2))
    score =np.power(score_2,0.2)+np.power(score_1,5)
 
   
    np.savetxt('排序分数.csv',score,delimiter=",")
    
    feature_number_list =list(range(5,21,1))+list(range(30,101,10))
    
    
    for feature_number in feature_number_list: 
        clf = RandomForestClassifier(n_estimators=200,max_features='sqrt', oob_score=True,random_state=120) 
        correct = 0
        for train, test in kf.split(X):
            idx = feature_ranking(score)  
            selected_features = X[:, idx[0:feature_number]] 
            clf.fit(selected_features[train], y[train])  
            y_predict = clf.predict(selected_features[test])
            acc = accuracy_score(y[test], y_predict) 
            correct = correct + acc
        acc_list_1.append(float(correct)/10)
        print('{:6f}'.format(float(correct)/10))
   # np.savetxt('xx.csv',np.array(acc_list_1),delimiter=',')
main1()